In [8]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module

ws = Workspace.get(name='kubeflow_ws_1', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='kubeflow-demo')

one2two = Module.from_yaml(ws, yaml_file='./noop/1in2out.spec.yaml')
one2one = Module.from_yaml(ws, yaml_file='./noop/1in1out.spec.yaml')
data = Dataset.get_by_name(ws, 'training_data')

In [9]:
@dsl.pipeline(
    name='2x Splits',
    description='A sample'
)
def cell_division():
    layer = 5
    nodes = []
    nodes.append(one2two(input1=data))
    last_layer = []
    for i in range(0, layer-1):
        print('i=', i, ' nodes len=', len(nodes))
        current_layer_nodes = []
        for j in range(0, pow(2,i)):
            print('j=', j)
            n = nodes[-j-1]
            current_layer_nodes.append(one2two(input1=n.outputs.output1))
            current_layer_nodes.append(one2two(input1=n.outputs.output2))
        nodes = nodes + current_layer_nodes
        last_layer = current_layer_nodes

    x = {}
    seq = 0
    for n in last_layer:
        seq += 1
        x['output{}'.format(seq)] = n.outputs.output1
        seq += 1
        x['output{}'.format(seq)] = n.outputs.output2
    return x

In [10]:
@dsl.pipeline(
    name='Chain',
    description='A sample'
)
def chain(input):
    length = 10
    last = None
    for i in range(0, length):
        j = None
        if last == None:
            j = input
        else:
            j = last.outputs.output1
        node = one2one(input1=j)
        last = node

    return {**last.outputs}

In [11]:
@dsl.pipeline(
    name='A pipeline composed with split and chain',
    description='A sample',
#    default_compute_target='cpu-lowpri'
    default_compute_target='kubeflow-aks'
)
def waterfall():
    part1 = cell_division()
    x = {}
    for o in part1.outputs.values():
        part2 = chain(o)
        x = {**x, **part2.outputs}

    return x

In [12]:
pipeline = waterfall()

pipeline.validate()

i= 0  nodes len= 1
j= 0
i= 1  nodes len= 3
j= 0
j= 1
i= 2  nodes len= 7
j= 0
j= 1
j= 2
j= 3
i= 3  nodes len= 15
j= 0
j= 1
j= 2
j= 3
j= 4
j= 5
j= 6
j= 7


<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_022c07a1-88a5-4bbe-8c08-c31ddc296208_widget', env_json='{"subscription…

{'result': 'validation passed', 'errors': []}

In [13]:
#pipeline.submit(experiment_name='samples').wait_for_completion()